# Interactive Tutorial: Basic Agent

This notebook provides an interactive introduction to the basic agent in DataMCPServerAgent. You'll learn how to:

1. Set up and configure the agent
2. Interact with the agent
3. Use special commands
4. Customize the agent's behavior

Let's get started!

## Setup

First, let's make sure we have the necessary imports and setup. We'll add the project root to the Python path to ensure we can import the required modules.

In [ ]:
import asyncio
import os
import sys

import ipywidgets as widgets
from IPython.display import clear_output, display

# Add the project root to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

# Import the agent

print(f"Project root: {project_root}")
print("Setup complete!")

## Configuration

Now, let's configure the agent. We'll create a configuration dictionary that specifies the agent's behavior.

In [ ]:
# Create a configuration dictionary
config = {
    "verbose": True,  # Enable verbose logging
    "memory_backend": "local",  # Use local memory backend
    "model": "claude-3-sonnet",  # Use Claude 3 Sonnet model
    "max_tokens": 4096  # Maximum number of tokens to generate
}

print("Configuration:")
for key, value in config.items():
    print(f"  {key}: {value}")

## Interactive Agent

Now, let's create an interactive interface for the agent. We'll use ipywidgets to create a text input, a button, and an output area.

In [ ]:
# Create a class to manage the agent's state
class AgentManager:
    def __init__(self, config):
        self.config = config
        self.conversation_history = []
        self.agent_running = False

    async def start_agent(self):
        """Start the agent."""
        self.agent_running = True
        return "Agent started. Type a message to begin."

    async def stop_agent(self):
        """Stop the agent."""
        self.agent_running = False
        return "Agent stopped."

    async def send_message(self, message):
        """Send a message to the agent."""
        if not self.agent_running:
            return "Agent is not running. Start the agent first."

        # Add the message to the conversation history
        self.conversation_history.append({"role": "user", "content": message})

        # Check for special commands
        if message.lower() in ["exit", "quit"]:
            await self.stop_agent()
            return "Agent stopped."

        # Process the message with the agent
        try:
            # In a real implementation, this would call the agent
            # For this example, we'll simulate the agent's response
            response = f"This is a simulated response to: {message}"

            # Add the response to the conversation history
            self.conversation_history.append({"role": "assistant", "content": response})

            return response
        except Exception as e:
            return f"Error: {str(e)}"

# Create an instance of the agent manager
agent_manager = AgentManager(config)

# Create widgets
text_input = widgets.Text(
    value='',
    placeholder='Enter your message',
    description='Message:',
    disabled=False,
    layout=widgets.Layout(width='80%')
)

output = widgets.Output(
    layout=widgets.Layout(border='1px solid black', width='100%', height='300px', overflow_y='auto')
)

send_button = widgets.Button(
    description='Send',
    disabled=False,
    button_style='',
    tooltip='Send message to agent',
    icon='paper-plane',
    layout=widgets.Layout(width='100px')
)

start_button = widgets.Button(
    description='Start Agent',
    disabled=False,
    button_style='success',
    tooltip='Start the agent',
    icon='play',
    layout=widgets.Layout(width='150px')
)

stop_button = widgets.Button(
    description='Stop Agent',
    disabled=False,
    button_style='danger',
    tooltip='Stop the agent',
    icon='stop',
    layout=widgets.Layout(width='150px')
)

clear_button = widgets.Button(
    description='Clear Output',
    disabled=False,
    button_style='',
    tooltip='Clear the output',
    icon='trash',
    layout=widgets.Layout(width='150px')
)

# Define button click handlers
async def on_send_button_clicked(b):
    message = text_input.value
    if not message:
        return

    with output:
        print(f"User: {message}")
        response = await agent_manager.send_message(message)
        print(f"Agent: {response}")

    text_input.value = ''

async def on_start_button_clicked(b):
    with output:
        response = await agent_manager.start_agent()
        print(f"System: {response}")

async def on_stop_button_clicked(b):
    with output:
        response = await agent_manager.stop_agent()
        print(f"System: {response}")

def on_clear_button_clicked(b):
    with output:
        clear_output()

# Use event loop to handle async functions
def handle_send_button_click(b):
    asyncio.create_task(on_send_button_clicked(b))

def handle_start_button_click(b):
    asyncio.create_task(on_start_button_clicked(b))

def handle_stop_button_click(b):
    asyncio.create_task(on_stop_button_clicked(b))

# Connect button click handlers
send_button.on_click(handle_send_button_click)
start_button.on_click(handle_start_button_click)
stop_button.on_click(handle_stop_button_click)
clear_button.on_click(on_clear_button_clicked)

# Handle Enter key in text input
def on_text_input_submit(widget):
    handle_send_button_click(None)

text_input.on_submit(on_text_input_submit)

# Create layout
input_box = widgets.HBox([text_input, send_button])
button_box = widgets.HBox([start_button, stop_button, clear_button])
vbox = widgets.VBox([button_box, output, input_box])

# Display widgets
display(vbox)

# Start the agent automatically
asyncio.create_task(on_start_button_clicked(None))

## Special Commands

The agent supports several special commands that you can use during a chat session:

- `exit` or `quit`: End the chat session
- `help`: Display available commands
- `clear`: Clear the chat history

Try using these commands in the interactive interface above.

## Customizing the Agent

You can customize the agent's behavior by modifying the configuration dictionary. Let's create a function to update the configuration and restart the agent.

In [ ]:
# Create widgets for configuration
verbose_checkbox = widgets.Checkbox(
    value=config["verbose"],
    description='Verbose',
    disabled=False
)

memory_backend_dropdown = widgets.Dropdown(
    options=['local', 'redis', 'mongodb'],
    value=config["memory_backend"],
    description='Memory Backend:',
    disabled=False
)

model_dropdown = widgets.Dropdown(
    options=['claude-3-sonnet', 'claude-3-opus', 'claude-3-haiku'],
    value=config["model"],
    description='Model:',
    disabled=False
)

max_tokens_slider = widgets.IntSlider(
    value=config["max_tokens"],
    min=1024,
    max=8192,
    step=1024,
    description='Max Tokens:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

update_button = widgets.Button(
    description='Update Configuration',
    disabled=False,
    button_style='info',
    tooltip='Update the agent configuration',
    icon='refresh'
)

config_output = widgets.Output(
    layout=widgets.Layout(border='1px solid black', width='100%', height='100px')
)

# Define button click handler
def on_update_button_clicked(b):
    # Update the configuration
    config["verbose"] = verbose_checkbox.value
    config["memory_backend"] = memory_backend_dropdown.value
    config["model"] = model_dropdown.value
    config["max_tokens"] = max_tokens_slider.value

    # Update the agent manager
    agent_manager.config = config

    # Display the updated configuration
    with config_output:
        config_output.clear_output()
        print("Updated Configuration:")
        for key, value in config.items():
            print(f"  {key}: {value}")

# Connect button click handler
update_button.on_click(on_update_button_clicked)

# Create layout
config_box = widgets.VBox([
    widgets.HBox([verbose_checkbox, memory_backend_dropdown]),
    widgets.HBox([model_dropdown, max_tokens_slider]),
    update_button,
    config_output
])

# Display widgets
display(config_box)

# Display initial configuration
with config_output:
    print("Current Configuration:")
    for key, value in config.items():
        print(f"  {key}: {value}")

## Conclusion

In this interactive tutorial, you've learned how to:

1. Set up and configure the basic agent
2. Interact with the agent using an interactive interface
3. Use special commands
4. Customize the agent's behavior

This is just a basic introduction to the DataMCPServerAgent. In the next tutorials, you'll learn about more advanced features like custom tools, memory management, and multi-agent systems.

Happy coding!